# Lungs assisted with mechanical ventilation: Pressure-controlled ventilation (PCV) mode.

In the following notebook we describe the methodology to simulate a PCV example using the approach proposal in Avilés & Hurtado (2022).

We begin by importing the necessary modules.

In [1]:
# Import and install 
try:
    import dolfin
    print("oui")
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

--2023-06-16 00:42:10--  https://fem-on-colab.github.io/releases/fenics-install.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.111.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4156 (4.1K) [application/x-sh]
Saving to: ‘/tmp/fenics-install.sh’

/tmp/fenics-install 100%[===================>]   4.06K  --.-KB/s    in 0s      

2023-06-16 00:42:10 (43.9 MB/s) - ‘/tmp/fenics-install.sh’ saved [4156/4156]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FENICS_INSTALLED=/usr/local/share/fem-on-colab/fenics.installed
+ [[ ! -f /usr/local/share/fem-on-colab/fenics.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/6895377/releases/pybind11-install.s

In [2]:
!pip install meshio==4.4.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 12.0 MB/s eta 0:00:00


In [3]:
!pip install pyDOE

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pyDOE: filename=pyDOE-0.3.8-py3-none-any.whl size=18167 sha256=1f2c0fd1b2c4ac5bd995989637bca0005f433f6c334cf213bdb17db9d6d60fdf
  Stored in directory: /root/.cache/pip/wheels/ce/b6/d7/c6b64746dba6433c593e471e0ac3acf4f36040456d1d160d17
Successfully built pyDOE


In [8]:
import meshio
import dolfin
import os
import matplotlib.pyplot as plt
import numpy as np
import time
import modelfunctions # manual import
import linearregression # manual import 
 
from ast import Interactive
from dolfin import *
from modelfunctions import solve_poroelasticity
from linearregression import regression
from sklearn import linear_model
from pyDOE import lhs

## Parameters dataset creation

1. We specify a `range` around the baseline values of the considered lung parameter values under analysis:

Lung tissue constitutive model parameters:
*   $c$
*   $\beta$
*   $c_1$
*   $c_3$

Lung permeability:
*   $k$

Spring stiffness (chest-wall effect):
*   $K_s$


\
2. We use Latin Hypercube Sampling to obtain `sample_num` number of samples of the parameter space, bounded in the specified range.  

In [9]:
# We consider a range of 50% around the mean
range = 50/100

# Constitutive model parameters bounds (Bir. 2019 et al.)
C_bir2019    = [-range*356.7 + 356.7 , 356.7 +range*356.7] # parameter c
Beta_bir2019 = [-range*1.075 + 1.075 , 1.075 +range*1.075] # parameter beta
C1_bir2019   = [-range*278.2 + 278.2 , 278.2 +range*278.2] # parameter c1
C3_bir2019   = [-range*5.766 + 5.766 , 5.766 +range*5.766] # parameter c3
                
# Mechanical parameters bounds
per = [-range*10000 + 10000 , 10000 +range*10000]          # parameter k
KKresortee = [-range*0.08 + 0.08 , 0.08 +range*0.08]       # parameter Ks

# We use Latin Hypercube Sampling approach to generate space-filling training samples.
# Number of samples
sample_num = 20
# Bounds
lb = np.array([C_bir2019[0], Beta_bir2019[0], C1_bir2019[0], C3_bir2019[0], per[0], KKresortee[0]])
ub = np.array([C_bir2019[1], Beta_bir2019[1], C1_bir2019[1], C3_bir2019[1], per[1], KKresortee[1]])
# Generation of samples
X_data = (ub-lb)*lhs(6, samples=sample_num) + lb

# We create an output vector to store the posterior simulation results (Crs and R)
Y_data = np.zeros([sample_num, 2])
print(X_data, Y_data)

[[4.72567798e+02 8.03318105e-01 1.97689886e+02 5.70984423e+00
  1.26810601e+04 5.46009340e-02]
 [3.53440877e+02 1.29376126e+00 2.25621783e+02 5.12849479e+00
  1.09438745e+04 1.18740980e-01]
 [2.98420620e+02 7.02389636e-01 2.15791894e+02 4.12123105e+00
  5.23096891e+03 1.02946187e-01]
 [4.08652636e+02 1.56815162e+00 1.76777883e+02 3.91553196e+00
  8.63183184e+03 1.12807007e-01]
 [1.89626669e+02 6.49295421e-01 3.12371391e+02 2.99866397e+00
  1.37263336e+04 1.07024797e-01]
 [4.31880003e+02 1.51189820e+00 3.98835783e+02 7.02681859e+00
  7.55876609e+03 4.06164140e-02]
 [4.60848793e+02 1.49921040e+00 3.40832262e+02 6.42723933e+00
  9.56702622e+03 6.20599628e-02]
 [5.19461244e+02 1.06311243e+00 3.28803953e+02 4.43961878e+00
  1.19233201e+04 9.74705990e-02]
 [3.75666769e+02 1.28076225e+00 4.12726999e+02 7.95657363e+00
  1.24579244e+04 8.00547506e-02]
 [4.25226692e+02 1.14072716e+00 3.76751699e+02 5.80979020e+00
  6.23581090e+03 9.29243430e-02]
 [2.81858230e+02 1.10478618e+00 1.65329840e+02 7.3

## Performing the simulations

Next, for each sample of model parameters, the problem is solved using finite elements in FEniCS. To this end we call `solve_poroelasticity`, in which the following steps are performed:

*   Load the mesh and boundaryes
*   Build function space. We use Taylor-Hood element
*   Initialize solver
*   Time-stepping loop

At the end of each simulation, we obtain arrays of pressure, airflow, and volume waveforms, along with a time array. 




In [10]:
# Measuring time
start_time = time.time()

# Simulation loop (it will execute 'sample_num' number of cases)
i = 0

while i < len(X_data):

    ii = i

    C_bir2019    = X_data[i,0] 
    Beta_bir2019 = X_data[i,1]
    C1_bir2019   = X_data[i,2]
    C3_bir2019   = X_data[i,3]
    per          = X_data[i,4]
    KKresortee   = X_data[i,5]
    
    models=['bir2019']
    for model in models:
        tiempos,Jacob,flux,presionestodas=solve_poroelasticity('TEST',model,'low',per,KKresortee,ii,C_bir2019,Beta_bir2019,C1_bir2019,C3_bir2019)
  
    i += 1

print("--- %s seconds ---" % (time.time() - start_time))

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_2269afe93cfdb70efbe7c05232b914e37e112d05

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00954294 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing representation of forms
INFO:FFC:  
INFO:FFC:Compiler stage 2 finished in 0.208689 seconds.

INFO:FFC:Compiler stage 3

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_b863195d8e0b681dc3527ba482c8cc38ad6593cf

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00881314 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing representation of forms
INFO:FFC:  
INFO:FFC:Compiler stage 2 finished in 0.0157831 seconds.

INFO:FFC:Compiler stage 3: Optimizing intermediate representati

Calling FFC just-in-time (JIT) compiler, this may take some time.


...

PRESSURE,TIME,MODEL: 0.0591 0.01 bir2019
PRESSURE,TIME,MODEL: 0.1182 0.02 bir2019
PRESSURE,TIME,MODEL: 0.1773 0.03 bir2019
PRESSURE,TIME,MODEL: 0.2364 0.04 bir2019
PRESSURE,TIME,MODEL: 0.2955 0.05 bir2019
PRESSURE,TIME,MODEL: 0.3546 0.06 bir2019
PRESSURE,TIME,MODEL: 0.4137 0.07 bir2019
PRESSURE,TIME,MODEL: 0.4728 0.08 bir2019
PRESSURE,TIME,MODEL: 0.5319 0.09 bir2019
PRESSURE,TIME,MODEL: 0.591 0.1 bir2019
PRESSURE,TIME,MODEL: 0.591 0.16 bir2019
PRESSURE,TIME,MODEL: 0.591 0.22000000000000003 bir2019
PRESSURE,TIME,MODEL: 0.591 0.28 bir2019
PRESSURE,TIME,MODEL: 0.591 0.34 bir2019
PRESSURE,TIME,MODEL: 0.591 0.4 bir2019
PRESSURE,TIME,MODEL: 0.591 0.4600000000000001 bir2019
PRESSURE,TIME,MODEL: 0.591 0.52 bir2019
PRESSURE,TIME,MODEL: 0.591 0.5800000000000001 bir2019
PRESSURE,TIME,MODEL: 0.591 0.64 bir2019
PRESSURE,TIME,MODEL: 0.591 0.7000000000000001 bir2019
PRESSURE,TIME,MODEL: 0.591 0.76 bir2019
PRESSURE,TIME,MODEL: 0.591 0.8200000000000001 bir2019
PRESSURE,TIME,MODEL: 0.591 0.88 bir2019
PR

Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling form ffc_form_71c6e048084bc625eb64c62f22ba24d82b9c1583

INFO:FFC:Compiler stage 1: Analyzing form(s)
INFO:FFC:-----------------------------------
DEBUG:FFC:  Preprocessing form using 'uflacs' representation family.
INFO:UFL:Adjusting missing element cell to tetrahedron.
INFO:UFL:Adjusting missing element cell to tetrahedron.
INFO:UFL:Adjusting missing element cell to tetrahedron.
INFO:UFL:Adjusting missing element cell to tetrahedron.
INFO:UFL:Adjusting missing element cell to tetrahedron.
INFO:FFC:  
INFO:FFC:  Geometric dimension:                 3
  Number of cell subdomains:           0
  Number of exterior_facet subdomains: 4
  Rank:                                1
  Arguments:                           '(v_0)'
  Number of coefficients:              8
  Coefficients:                        '[f_25314, f_25315, f_25318, f_25327, f_25328
                                       , f_25329,

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling form ffc_form_b03949cca60b474770c26101d2ff791e32ce05ce

INFO:FFC:Compiler stage 1: Analyzing form(s)
INFO:FFC:-----------------------------------
DEBUG:FFC:  Preprocessing form using 'uflacs' representation family.
INFO:UFL:Adjusting missing element cell to tetrahedron.
INFO:UFL:Adjusting missing element cell to tetrahedron.
INFO:UFL:Adjusting missing element cell to tetrahedron.
INFO:UFL:Adjusting missing element cell to tetrahedron.
INFO:UFL:Adjusting missing element cell to tetrahedron.
INFO:FFC:  
INFO:FFC:  Geometric dimension:                 3
  Number of cell subdomains:           0
  Number of exterior_facet subdomains: 4
  Rank:                                2
  Arguments:                           '(v_0, v_1)'
  Number of coefficients:              8
  Coefficients:                        '[f_25314, f_25315, f_25318, f_25327, f_25328
                                       , f_2

PRESSURE,TIME,MODEL: 0.0591 0.01 bir2019
PRESSURE,TIME,MODEL: 0.1182 0.02 bir2019
PRESSURE,TIME,MODEL: 0.1773 0.03 bir2019
PRESSURE,TIME,MODEL: 0.2364 0.04 bir2019
PRESSURE,TIME,MODEL: 0.2955 0.05 bir2019
PRESSURE,TIME,MODEL: 0.3546 0.06 bir2019
PRESSURE,TIME,MODEL: 0.4137 0.07 bir2019
PRESSURE,TIME,MODEL: 0.4728 0.08 bir2019
PRESSURE,TIME,MODEL: 0.5319 0.09 bir2019
PRESSURE,TIME,MODEL: 0.591 0.1 bir2019
PRESSURE,TIME,MODEL: 0.591 0.16 bir2019
PRESSURE,TIME,MODEL: 0.591 0.22000000000000003 bir2019
PRESSURE,TIME,MODEL: 0.591 0.28 bir2019
PRESSURE,TIME,MODEL: 0.591 0.34 bir2019
PRESSURE,TIME,MODEL: 0.591 0.4 bir2019
PRESSURE,TIME,MODEL: 0.591 0.4600000000000001 bir2019
PRESSURE,TIME,MODEL: 0.591 0.52 bir2019
PRESSURE,TIME,MODEL: 0.591 0.5800000000000001 bir2019
PRESSURE,TIME,MODEL: 0.591 0.64 bir2019
PRESSURE,TIME,MODEL: 0.591 0.7000000000000001 bir2019
PRESSURE,TIME,MODEL: 0.591 0.76 bir2019
PRESSURE,TIME,MODEL: 0.591 0.8200000000000001 bir2019
PRESSURE,TIME,MODEL: 0.591 0.88 bir2019
PR

## Obtaining respiratory-system compliance and airways resistance

Finally, from the ventilator signals and using the single compartment equation of motion, we obtain the respiratory-system compliance and airways resistance parameters via least-squares fitting by calling the `regression` function.

In [11]:
# Simulation loop (it will execute 'sample_num' number of cases)
i = 0

while i < len(X_data):

    ii = i

    # Debe existir una carpeta llamada derecho donde estén las curvas del pulmón derecho. Idem para el izquierdo
    name='bir2019'

    ## We load the mechanical ventilation curves (considering both lungs) 
    fluxes=fileflujos=np.load(name+str(ii)+'fluxes.npy')*8+np.load(name+str(ii)+'fluxes.npy')*8#[0:84]
    presionestodas=filepresiones=np.load(name+str(ii)+'presionestodas.npy')#+filepresionesi
    tiempos=filetiempos=np.load(name+str(ii)+'tiempos.npy')
    Jacob=filevolumenes=np.load(name+str(ii)+'volumenes.npy')*8+np.load(name+str(ii)+'volumenes.npy')*8#[0:84]
    Jacob=Jacob-Jacob[0]

    # Leemos npy generado de ambos pulmones
    maflujos=fluxes*60
    mapresiones=presionestodas
    matiempos=tiempos
    mavolumenes=Jacob

    # Agregamos punto 0
    maflujos=np.concatenate((np.array([0]),np.asarray(maflujos)))
    mapresiones=np.concatenate((np.array([0]),np.asarray(mapresiones)))
    matiempos=np.concatenate((np.array([0]),np.asarray(matiempos)))
    mavolumenes=np.concatenate((np.array([0]),np.asarray(mavolumenes)))


    regression(maflujos,mapresiones,matiempos,mavolumenes,name,Y_data,i)

    i += 1

bir2019
Resistance(R)= (3.203457629003445, 2) cm H2O L/S, 
Compliance (Crs)= 112.43723232711235 ml/cm H2O
---------------
bir2019
Resistance(R)= (3.783915048545522, 2) cm H2O L/S, 
Compliance (Crs)= 69.73503104979156 ml/cm H2O
---------------
bir2019
Resistance(R)= (7.596405352575292, 2) cm H2O L/S, 
Compliance (Crs)= 80.34245589528902 ml/cm H2O
---------------
bir2019
Resistance(R)= (4.766424120757759, 2) cm H2O L/S, 
Compliance (Crs)= 61.74669913105019 ml/cm H2O
---------------
bir2019
Resistance(R)= (3.003229366259611, 2) cm H2O L/S, 
Compliance (Crs)= 106.58159073644714 ml/cm H2O
---------------
bir2019
Resistance(R)= (5.323488079650627, 2) cm H2O L/S, 
Compliance (Crs)= 93.31997858415096 ml/cm H2O
---------------
bir2019
Resistance(R)= (4.262304962724921, 2) cm H2O L/S, 
Compliance (Crs)= 79.34580470189096 ml/cm H2O
---------------
bir2019
Resistance(R)= (3.454309556453954, 2) cm H2O L/S, 
Compliance (Crs)= 69.66710935348667 ml/cm H2O
---------------
bir2019
Resistance(R)= (3.3079

## Display the dataset

Finally, we display the created dataset. From this, it can be observed that:

- `X_data` is an array that corresponds to the sampled lung parameters using the Latin hypercube design: each row a sample containing the values of the six parameters.

- `Y_data` is an array where a single row corresponds to the obtained respiratory-system compliance and airways resistance after simulating the corresponding sample of lung model parameters.

In [12]:
print(X_data)
print(Y_data)

np.save('input_data.npy', X_data)
np.save('output_data.npy', Y_data)

[[4.72567798e+02 8.03318105e-01 1.97689886e+02 5.70984423e+00
  1.26810601e+04 5.46009340e-02]
 [3.53440877e+02 1.29376126e+00 2.25621783e+02 5.12849479e+00
  1.09438745e+04 1.18740980e-01]
 [2.98420620e+02 7.02389636e-01 2.15791894e+02 4.12123105e+00
  5.23096891e+03 1.02946187e-01]
 [4.08652636e+02 1.56815162e+00 1.76777883e+02 3.91553196e+00
  8.63183184e+03 1.12807007e-01]
 [1.89626669e+02 6.49295421e-01 3.12371391e+02 2.99866397e+00
  1.37263336e+04 1.07024797e-01]
 [4.31880003e+02 1.51189820e+00 3.98835783e+02 7.02681859e+00
  7.55876609e+03 4.06164140e-02]
 [4.60848793e+02 1.49921040e+00 3.40832262e+02 6.42723933e+00
  9.56702622e+03 6.20599628e-02]
 [5.19461244e+02 1.06311243e+00 3.28803953e+02 4.43961878e+00
  1.19233201e+04 9.74705990e-02]
 [3.75666769e+02 1.28076225e+00 4.12726999e+02 7.95657363e+00
  1.24579244e+04 8.00547506e-02]
 [4.25226692e+02 1.14072716e+00 3.76751699e+02 5.80979020e+00
  6.23581090e+03 9.29243430e-02]
 [2.81858230e+02 1.10478618e+00 1.65329840e+02 7.3